# Import Modules

In [2]:
import pandas as pd
from sklearn import preprocessing
import torch
from torch import nn
from torch import optim

import utils

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare Data

In [7]:
source_X = pd.read_csv("./deep_occupancy_detection/data/4_X_train.csv").values
target_X = pd.read_csv("./deep_occupancy_detection/data/2_X_train.csv").values
source_y_task = pd.read_csv("./deep_occupancy_detection/data/4_Y_train.csv").values.reshape(-1)
target_y_task = pd.read_csv("./deep_occupancy_detection/data/2_Y_train.csv").values.reshape(-1)

scaler = preprocessing.StandardScaler()
source_X = scaler.fit_transform(source_X)
target_X = scaler.fit_transform(target_X)

In [8]:
source_X, source_y_task = utils.apply_sliding_window(source_X, source_y_task, filter_len=3)
target_X, target_y_task = utils.apply_sliding_window(target_X, target_y_task, filter_len=3)

train_source_X, _, _, train_source_y_task, _, _ = utils.train_val_test_split(source_X, source_y_task)
train_target_X, _, test_target_X, train_target_y_task, _, test_target_y_task = utils.train_val_test_split(target_X, target_y_task)
# TODO: test_target_X, test_target_y_task should be used in evaluation

In [9]:
source_loader, target_loader, train_source_y_task, train_source_X, train_target_X, train_target_y_task = utils.get_loader(train_source_X, train_target_X, train_source_y_task, train_target_y_task)

# Instantiate Feature Extractor, Domain Classifier, Task Classifier

In [18]:
class ManyToOneRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    def forward(self, x):
        _, (x, _) = self.rnn(x)
        return x

In [19]:
rnn = ManyToOneRNN(input_size=13, hidden_size=10, num_layers=1).to(device)

In [25]:
train_source_X.shape

torch.Size([458, 3, 13])

In [24]:
rnn(train_source_X).shape

torch.Size([1, 458, 10])